# [Module 5.1] Batch Inference Job 실행하기

이 노트북은 주로 아래와 같은 작업을 합니다.
* batch inference inpput json파일 작성
* inpput json파일을 S3에 업로드
* batch inference job 실행
* batch inference job 실행 결과 보기


In [1]:
import boto3
import json
import numpy as np
import pandas as pd
import time
from datetime import datetime

다음으로 여러분의 환경이 Amazon Personalize와 성공적으로 통신할 수 있는지 확인해야 합니다.

In [2]:
# Configure the SDK to Personalize:
personalize = boto3.client('personalize')

아래 코드 셀은 이전 notebook에서 저장했던 공유 변수들을 불러옵니다.

In [3]:
%store -r

## Help 함수 만들기

아래는 item_id의 영화 제목을 제공함는 함수를 만듭니다.

In [4]:
items_all = pd.read_csv('./ml-1m/movies.dat',sep='::', encoding='latin1',names=['ITEM_ID', 'TITLE', 'GENRE'],)
items=items_all.copy()

items['to_keep'] = items['ITEM_ID'].apply(lambda x:x in unique_items)
items=items[items['to_keep']]
del items['to_keep']
items.tail()

#len(unique_items)

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/ipykernel/__main__.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  if __name__ == '__main__':


,ITEM_ID,TITLE,GENRE
3878,3948,Meet the Parents (2000),Comedy
3879,3949,Requiem for a Dream (2000),Drama
3880,3950,Tigerland (2000),Drama
3881,3951,Two Family House (2000),Drama
3882,3952,"Contender, The (2000)",Drama|Thriller


In [5]:
def get_movie_title(movie_id):
    """
    Takes in an ID, returns a title
    """
    movie_id = int(movie_id)
    movie_title=items[items['ITEM_ID']==movie_id]['TITLE']
    return (movie_title.tolist())


## Amazon Personalize Batch Export 작업 


Amazon Personalize Batch기능을 활용하려면 json 파일 형식으로 추천 받고하 자는 사용자 또는 아이템 아이디를 json 형태의 파일로 s3에 저장하여야 합니다. Output의 형식도 json형태로 저장되며 지정한 S3 bucket 경로에 저장 되게 됩니다. 

HRNN 솔루션  Batch Input 예제: 

```JSON,
    {"userId": "4638"},
    {"userId": "663"},
    {"userId": "3384"},
```


Batch Output 예제: 
```JSON,
{"input":{"userId":"4638"}, "output": {"recommendedItems": ["296", "1", "260", "318"]}}
{"input":{"userId":"663"}, "output": {"recommendedItems": ["1393", "3793", "2701", "3826"]}}
{"input":{"userId":"3384"}, "output": {"recommendedItems": ["8368", "5989", "40815", "48780"]}}
```


#### input json 파일 준비

In [6]:
#Get the user list
df=pd.read_csv(warm_train_interaction_filename)
df_users = df['USER_ID'].unique()
df_users=pd.DataFrame(df_users,columns=['USER_ID'])


In [7]:
#Get the user list
#batch_users = df_users.sample(3).index.tolist()
batch_users=df_users.index.tolist()
data_dir="dataset/"
# Write the file to disk
json_input_filename = "json_input.json"
with open(data_dir+json_input_filename, 'w') as json_input:
    for user_id in batch_users:
        json_input.write('{"userId": "' + str(user_id) + '"}\n')

In [8]:
# Showcase the input file:
!head -n5 $data_dir$json_input_filename

{"userId": "0"}
{"userId": "1"}
{"userId": "2"}
{"userId": "3"}
{"userId": "4"}


In [9]:
!tail -n5 $data_dir$json_input_filename

{"userId": "6035"}
{"userId": "6036"}
{"userId": "6037"}
{"userId": "6038"}
{"userId": "6039"}


#### S3에 업로드

In [10]:
# Upload files to S3
boto3.Session().resource('s3').Bucket(bucket).Object(data_dir+json_input_filename).upload_file(data_dir+json_input_filename)
s3_input_path = "s3://" + bucket + "/" + data_dir+json_input_filename
print(s3_input_path)

s3://sagemaker-ap-northeast-2-057716757052/dataset/json_input.json


In [11]:
# Define the output path
s3_output_path = "s3://" + bucket + "/"+data_dir
print(s3_output_path)

s3://sagemaker-ap-northeast-2-057716757052/dataset/


In [12]:
print(role_arn)

arn:aws:iam::057716757052:role/PersonalizeRoleDemo60498


#### batch inference job 실행

In [13]:
batchInferenceJobArn = personalize.create_batch_inference_job (
    solutionVersionArn = hrnn_solution_version_arn,
    jobName = "POC-Batch-Inference-Job-HRNN-"+suffix,
    roleArn = role_arn,
    jobInput = 
     {"s3DataSource": {"path": s3_input_path}},
    jobOutput = 
     {"s3DataDestination":{"path": s3_output_path}}
)
batchInferenceJobArn = batchInferenceJobArn['batchInferenceJobArn']

In [14]:
current_time = datetime.now()
print("Import Started on: ", current_time.strftime("%I:%M:%S %p"))

max_time = time.time() + 3*60*60 # 3 hours
while time.time() < max_time:
    describe_dataset_inference_job_response = personalize.describe_batch_inference_job(
        batchInferenceJobArn = batchInferenceJobArn
    )
    status = describe_dataset_inference_job_response["batchInferenceJob"]['status']
    print("DatasetInferenceJob: {}".format(status))
    
    if status == "ACTIVE" or status == "CREATE FAILED":
        break
        
    time.sleep(60)
    
current_time = datetime.now()
print("Import Completed on: ", current_time.strftime("%I:%M:%S %p"))

Import Started on:  08:23:11 AM
DatasetInferenceJob: CREATE PENDING
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInferenceJob: CREATE IN_PROGRESS
DatasetInfer

## Batch Transform 의 S3 결과 확인하기

여기서는 테스트로 10명의 유저의 결과를 로딩 합니다.

In [15]:
s3 = boto3.client('s3')
export_name = json_input_filename + ".out"
s3.download_file(bucket,data_dir+export_name,data_dir+export_name)

# Update DF rendering
pd.set_option('display.max_rows', 30)
with open("dataset/"+export_name) as json_file:
    # Get the first line and parse it
    line = json.loads(json_file.readline())
    # Do the same for the other lines
    
    # 유저의 로딩 수를 제한함.
    num_load_user_id = 10
    num_load = 0
    
    if 'bulk_recommendations_df' in locals():
        del bulk_recommendations_df
    
    while line:
        # extract the user ID 
        col_header = "User: " + line['input']['userId']
        # Create a list for all the artists
        recommendation_list = []
        # Add all the entries
        for item in line['output']['recommendedItems']:
            title = get_movie_title(item)
            recommendation_list.append(title)
        if 'bulk_recommendations_df' in locals():
            new_rec_DF = pd.DataFrame(recommendation_list, columns = [col_header])
            bulk_recommendations_df = bulk_recommendations_df.join(new_rec_DF)
        else:
            bulk_recommendations_df = pd.DataFrame(recommendation_list, columns=[col_header])

        try:
            line = json.loads(json_file.readline())
        except:
            line = None
            
        num_load = num_load + 1            
        if num_load > num_load_user_id:
            break
bulk_recommendations_df

,User: 1024,User: 1029,User: 1064,User: 1080,User: 110,User: 1102,User: 1134,User: 1173,User: 1207,User: 1232,User: 1233
0,Four Weddings and a Funeral (1994),Weird Science (1985),G.I. Jane (1997),Citizen Kane (1941),Mission to Mars (2000),"Wizard of Oz, The (1939)",Crocodile Dundee (1986),Backdraft (1991),Rosemary's Baby (1968),Four Weddings and a Funeral (1994),"Rocky Horror Picture Show, The (1975)"
1,Romancing the Stone (1984),Crocodile Dundee (1986),"Devil's Own, The (1997)","Face in the Crowd, A (1957)",Pitch Black (2000),To Kill a Mockingbird (1962),Young Sherlock Holmes (1985),Natural Born Killers (1994),Ghostbusters (1984),"Purple Rose of Cairo, The (1985)",Back to the Future Part II (1989)
2,"English Patient, The (1996)",Adventures in Babysitting (1987),It Could Happen to You (1994),"400 Blows, The (Les Quatre cents coups) (1959)",Where the Heart Is (2000),Amadeus (1984),"Goonies, The (1985)",Conspiracy Theory (1997),"American Werewolf in London, An (1981)",Pleasantville (1998),Apollo 13 (1995)
3,"Fabulous Baker Boys, The (1989)",Night Shift (1982),Outbreak (1995),To Kill a Mockingbird (1962),Grumpier Old Men (1995),"Grapes of Wrath, The (1940)",Mighty Joe Young (1949),Broken Arrow (1996),Ferris Bueller's Day Off (1986),Big (1988),"War of the Worlds, The (1953)"
4,Sabrina (1954),Working Girl (1988),Powder (1995),Midnight Cowboy (1969),Scream 3 (2000),Apocalypse Now (1979),"Jewel of the Nile, The (1985)","Crow, The (1994)",Airplane! (1980),Grosse Pointe Blank (1997),Braveheart (1995)
5,Forrest Gump (1994),History of the World: Part I (1981),Evita (1996),Amadeus (1984),She's the One (1996),"Godfather: Part II, The (1974)",Escape from New York (1981),Fallen (1998),Fatal Attraction (1987),Mighty Aphrodite (1995),As Good As It Gets (1997)
6,Speed (1994),"Man with Two Brains, The (1983)",One Fine Day (1996),Raiders of the Lost Ark (1981),Reindeer Games (2000),"Manchurian Candidate, The (1962)","Journey of Natty Gann, The (1985)",Air Force One (1997),"Shining, The (1980)",Moonstruck (1987),Akira (1988)
7,Titanic (1997),Arthur (1981),Far and Away (1992),Marty (1955),Titan A.E. (2000),Raiders of the Lost Ark (1981),Indiana Jones and the Temple of Doom (1984),Outbreak (1995),Hoosiers (1986),Waking Ned Devine (1998),Galaxy Quest (1999)
8,"Goodbye Girl, The (1977)",Back to the Future Part II (1989),Patch Adams (1998),Chinatown (1974),Space Jam (1996),Chinatown (1974),"NeverEnding Story, The (1984)",Die Hard: With a Vengeance (1995),Amadeus (1984),Risky Business (1983),Starman (1984)
9,Indochine (1992),Back to School (1986),Practical Magic (1998),"Godfather: Part II, The (1974)",Raiders of the Lost Ark (1981),Rocky IV (1985),Labyrinth (1986),Executive Decision (1996),"Exorcist, The (1973)","Gods Must Be Crazy, The (1980)",Time Bandits (1981)
